In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
n_epochs = 5 # 模型训练5轮
log_interval = 30 #控制打印频率的，设n = 30*batch_size，即n张图后打印一次进度
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 根据设备是否支持GPU来选择硬件 
size = 32 # 对输入图片进行处理，拉伸为32*32的图片，这是为了复刻手写数字识别的神经网络，其输入为32*32的灰度图像
learn_rate = 0.03 # 学习率
momentum = 0.1  # 动量

In [3]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz
from torchvision.datasets import MNIST
transform = transforms.Compose(
    [ transforms.Resize(size), transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))]) # 正则化处理，相当于z-score
trainset = MNIST(root = './', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testset = MNIST(root = './', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1000, shuffle=True, num_workers=2)

# classes = ('1', '2', '3', '4', '5', '6', '7', '8', '9', '0')

'wget' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
tar: Error opening archive: Failed to open 'MNIST.tar.gz'


In [4]:
examples = enumerate(testloader)
batch_idx, (example_data, example_targets) = next(examples)
print(example_targets)
print(example_data.shape)

tensor([5, 6, 3, 5, 3, 8, 6, 8, 4, 3, 9, 1, 4, 9, 9, 4, 7, 3, 8, 9, 6, 3, 8, 0,
        2, 9, 4, 1, 4, 8, 8, 4, 7, 7, 5, 0, 4, 1, 8, 3, 7, 6, 4, 6, 3, 9, 9, 0,
        2, 1, 9, 0, 2, 6, 7, 6, 3, 9, 7, 6, 3, 0, 1, 2, 8, 5, 1, 3, 0, 8, 4, 4,
        0, 4, 6, 5, 1, 0, 7, 5, 6, 8, 1, 4, 0, 2, 7, 7, 2, 6, 3, 9, 5, 2, 5, 5,
        0, 7, 3, 7, 8, 5, 0, 3, 1, 1, 9, 0, 9, 1, 6, 3, 5, 9, 1, 0, 7, 8, 8, 5,
        3, 7, 8, 7, 3, 9, 4, 8, 5, 8, 4, 3, 1, 7, 4, 1, 8, 1, 3, 2, 3, 2, 0, 7,
        6, 3, 4, 0, 3, 8, 6, 3, 9, 0, 0, 1, 9, 1, 4, 7, 9, 5, 4, 2, 1, 6, 6, 2,
        8, 8, 2, 9, 5, 2, 8, 3, 8, 5, 8, 8, 5, 9, 3, 5, 2, 3, 2, 7, 3, 4, 0, 6,
        8, 5, 0, 9, 7, 8, 5, 0, 4, 8, 6, 1, 3, 2, 2, 0, 6, 6, 1, 9, 4, 1, 1, 4,
        2, 4, 1, 5, 8, 7, 9, 2, 2, 1, 1, 3, 5, 0, 0, 4, 6, 5, 2, 9, 9, 4, 8, 5,
        9, 9, 9, 0, 2, 1, 5, 1, 4, 5, 6, 6, 0, 5, 8, 7, 8, 1, 6, 9, 5, 4, 2, 2,
        4, 3, 3, 6, 4, 4, 4, 0, 3, 1, 5, 5, 8, 0, 3, 9, 2, 8, 4, 4, 4, 7, 7, 8,
        0, 1, 8, 1, 8, 1, 8, 9, 7, 4, 7,

In [5]:
import matplotlib.pyplot as plt
fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
  plt.title("Ground Truth: {}".format(example_targets[i]))
  plt.xticks([])
  plt.yticks([])
plt.show()

: 

In [ ]:
network = Net().to(DEVICE)
optimizer = optim.SGD(network.parameters(), lr = learn_rate, momentum=momentum) # 学习率，动量

In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(trainloader.dataset) for i in range(n_epochs + 1)] #test_losses为横坐标，test_losses为纵坐标
test_acc = []

In [ ]:
def train(epoch, device):
  network.train() # 调用上一步实例化对象network中的方法（该方法包内已经写好）
  for batch_idx, (data, target) in enumerate(trainloader): # 按batch_size为集合对象进行逐个处理
    data, target = data.to(device), target.to(device) # data是图片，target是标签，device是为了有GPU情况下使用GPU加速
    optimizer.zero_grad() # 开始进行BP之前将梯度设置为零，因为PyTorch会在随后的BP中累积梯度
    output = network(data) 
    loss = F.nll_loss(output, target) # 函数全称是negative log likelihood loss，下面博客有详细解释
    # https://blog.csdn.net/weixin_38145317/article/details/103288032
    loss.backward() # 根据误差进行BP
    optimizer.step()
    if batch_idx % log_interval == 0: # 控制输出频率
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(trainloader.dataset),
        100. * batch_idx / len(trainloader), loss.item()))
      train_losses.append(loss.item()) # 记录并储存train loss
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(trainloader.dataset)))

In [ ]:
def test(device):
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in testloader:
      data, target = data.to(device), target.to(device)
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1] # 找到概率最大的下标，为预测类别
      correct += pred.eq(target.data.view_as(pred)).sum() # x下面都是记录数据用于绘图，不再解释
  test_loss /= len(testloader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(testloader.dataset),
    100. * correct / len(testloader.dataset)))
  test_acc.append(correct / len(testloader.dataset))

In [ ]:
test(DEVICE)
for epoch in range(1, n_epochs + 1):
  train(epoch, DEVICE)
  test(DEVICE)
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue') # train_losses变化曲线
plt.plot(test_counter, test_acc, color='yellow') # test集accuracy变化曲线
plt.scatter(test_counter, test_losses, color='red') # test集loss散点图
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss and test accuarcy')
plt.show()

In [ ]:
examples = enumerate(testloader)
batch_idx, (example_data, example_targets) = next(examples)
with torch.no_grad():
  example_data, example_targets = example_data.to(DEVICE), example_targets.to(DEVICE)
  output = network(example_data)
fig = plt.figure()
for i in range(6):
  plt.subplot(2,3,i+1)
  plt.tight_layout()
  plt.imshow(example_data[i][0].cpu().clone().numpy(), cmap='gray', interpolation='none')
  plt.title("Prediction: {}".format(
    output.data.max(1, keepdim=True)[1][i].item()))
  plt.xticks([])
  plt.yticks([])
plt.show()